In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
data = pd.read_csv("/kaggle/input/customer-churn-dataset/customer_churn_dataset-training-master.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440833 entries, 0 to 440832
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CustomerID         440832 non-null  float64
 1   Age                440832 non-null  float64
 2   Gender             440832 non-null  object 
 3   Tenure             440832 non-null  float64
 4   Usage Frequency    440832 non-null  float64
 5   Support Calls      440832 non-null  float64
 6   Payment Delay      440832 non-null  float64
 7   Subscription Type  440832 non-null  object 
 8   Contract Length    440832 non-null  object 
 9   Total Spend        440832 non-null  float64
 10  Last Interaction   440832 non-null  float64
 11  Churn              440832 non-null  float64
dtypes: float64(9), object(3)
memory usage: 40.4+ MB


In [4]:
data.isnull().sum()

CustomerID           1
Age                  1
Gender               1
Tenure               1
Usage Frequency      1
Support Calls        1
Payment Delay        1
Subscription Type    1
Contract Length      1
Total Spend          1
Last Interaction     1
Churn                1
dtype: int64

In [5]:
data.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,1.0


In [6]:
data['Churn'].value_counts()

1.0    249999
0.0    190833
Name: Churn, dtype: int64

In [7]:
data = data.dropna()

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440832 entries, 0 to 440832
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CustomerID         440832 non-null  float64
 1   Age                440832 non-null  float64
 2   Gender             440832 non-null  object 
 3   Tenure             440832 non-null  float64
 4   Usage Frequency    440832 non-null  float64
 5   Support Calls      440832 non-null  float64
 6   Payment Delay      440832 non-null  float64
 7   Subscription Type  440832 non-null  object 
 8   Contract Length    440832 non-null  object 
 9   Total Spend        440832 non-null  float64
 10  Last Interaction   440832 non-null  float64
 11  Churn              440832 non-null  float64
dtypes: float64(9), object(3)
memory usage: 43.7+ MB


In [9]:
data.columns

Index(['CustomerID', 'Age', 'Gender', 'Tenure', 'Usage Frequency',
       'Support Calls', 'Payment Delay', 'Subscription Type',
       'Contract Length', 'Total Spend', 'Last Interaction', 'Churn'],
      dtype='object')

In [10]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
data[['Gender','Subscription Type','Contract Length']] = encoder.fit_transform(data[['Gender','Subscription Type','Contract Length']])

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440832 entries, 0 to 440832
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CustomerID         440832 non-null  float64
 1   Age                440832 non-null  float64
 2   Gender             440832 non-null  float64
 3   Tenure             440832 non-null  float64
 4   Usage Frequency    440832 non-null  float64
 5   Support Calls      440832 non-null  float64
 6   Payment Delay      440832 non-null  float64
 7   Subscription Type  440832 non-null  float64
 8   Contract Length    440832 non-null  float64
 9   Total Spend        440832 non-null  float64
 10  Last Interaction   440832 non-null  float64
 11  Churn              440832 non-null  float64
dtypes: float64(12)
memory usage: 43.7 MB


In [12]:
data = data.drop(['CustomerID'],axis=1)

In [13]:
X = data.drop(['Churn'],axis=1)
y = data['Churn']

In [14]:
from sklearn.model_selection import train_test_split
X_train , X_valid , y_train ,y_valid =  train_test_split(X , y , test_size=0.1 , random_state=42)

In [15]:
from functools import partial

RDense = partial(tf.keras.layers.Dense , activation="selu" , kernel_initializer = "lecun_normal")
Dropout = partial(tf.keras.layers.AlphaDropout)
Normalization = partial(tf.keras.layers.BatchNormalization)
output =  partial(tf.keras.layers.Dense , activation="softmax")

In [16]:
early_stopping = tf.keras.callbacks.EarlyStopping(
min_delta = 0.001 , patience = 20 , restore_best_weights = True )

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    Normalization(),
    Dropout(rate=0.2),
    RDense(100),
    Normalization(),
    Dropout(rate=0.2),
    RDense(50),
    Normalization(),
    Dropout(rate=0.2),
    output(2)
])

In [18]:
model.compile(loss="sparse_categorical_crossentropy" , optimizer=tf.keras.optimizers.SGD(learning_rate=0.01 , momentum = 0.9) , metrics=["accuracy"])

In [19]:
history = model.fit(X_train , y_train , epochs = 10 , validation_data = (X_valid , y_valid) , callbacks=[early_stopping])

Epoch 1/10
12399/12399 [==============================] - 65s 5ms/step - loss: 0.3605 - accuracy: 0.8361 - val_loss: 0.3113 - val_accuracy: 0.9127
Epoch 2/10
12399/12399 [==============================] - 57s 5ms/step - loss: 0.3128 - accuracy: 0.8647 - val_loss: 0.2824 - val_accuracy: 0.9213
Epoch 3/10
12399/12399 [==============================] - 57s 5ms/step - loss: 0.3027 - accuracy: 0.8710 - val_loss: 0.2698 - val_accuracy: 0.9288
Epoch 4/10
12399/12399 [==============================] - 57s 5ms/step - loss: 0.2961 - accuracy: 0.8741 - val_loss: 0.2623 - val_accuracy: 0.9261
Epoch 5/10
12399/12399 [==============================] - 56s 5ms/step - loss: 0.2943 - accuracy: 0.8748 - val_loss: 0.2713 - val_accuracy: 0.9296
Epoch 6/10
12399/12399 [==============================] - 57s 5ms/step - loss: 0.2934 - accuracy: 0.8757 - val_loss: 0.3069 - val_accuracy: 0.9293
Epoch 7/10
12399/12399 [==============================] - 56s 5ms/step - loss: 0.2910 - accuracy: 0.8765 - val_loss: 0

In [20]:
test = pd.read_csv("/kaggle/input/customer-churn-dataset/customer_churn_dataset-testing-master.csv")

In [21]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64374 entries, 0 to 64373
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CustomerID         64374 non-null  int64 
 1   Age                64374 non-null  int64 
 2   Gender             64374 non-null  object
 3   Tenure             64374 non-null  int64 
 4   Usage Frequency    64374 non-null  int64 
 5   Support Calls      64374 non-null  int64 
 6   Payment Delay      64374 non-null  int64 
 7   Subscription Type  64374 non-null  object
 8   Contract Length    64374 non-null  object
 9   Total Spend        64374 non-null  int64 
 10  Last Interaction   64374 non-null  int64 
 11  Churn              64374 non-null  int64 
dtypes: int64(9), object(3)
memory usage: 5.9+ MB


In [22]:
encoder = OrdinalEncoder()
test[['Gender','Subscription Type','Contract Length']] = encoder.fit_transform(test[['Gender','Subscription Type','Contract Length']])

In [23]:
test = test.drop(['CustomerID'],axis=1)

In [24]:
X = test.drop(['Churn'],axis=1)
y = test['Churn']

In [25]:
model.evaluate(X , y)

2012/2012 [==============================] - 5s 2ms/step - loss: 5.5384 - accuracy: 0.5668


[5.5383734703063965, 0.5668126940727234]